# Time Frequency Analysis of Percept recordings

## 0. Loading packages and functions, defining paths

In [2]:
# Importing Python and external packages
import os
import sys
import importlib
import json
from dataclasses import dataclass, field, fields
from itertools import compress
import csv
import pandas as pd
import numpy as np

import scipy
import matplotlib.pyplot as plt
from scipy import signal

import openpyxl
from openpyxl import Workbook, load_workbook
import xlrd

#mne
import mne_bids
import mne


# additionally imported from Varvara
# from importlib import reload          # is this necessary?

# from scipy.signal import spectrogram, hanning     # is this necessary?

# from mne.time_frequency import tfr_morlet         # is this necessary?

In [3]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('mne_bids', mne_bids.__version__)
print('mne', mne.__version__)
print('sci-py', scipy.__version__)

Python sys 3.10.6 | packaged by conda-forge | (main, Oct 24 2022, 16:02:16) [MSC v.1916 64 bit (AMD64)]
pandas 1.4.4
numpy 1.23.3
mne_bids 0.11.1
mne 1.2.1
sci-py 1.9.3


In [4]:
# create a path to the PyPerceive_Project folder -> project_path
# and a path to the "Data" folder and "Code" folder within the project
project_path = os.getcwd()
while project_path[-18:] != 'PyPerceive_Project':
    project_path = os.path.dirname(project_path)
    
data_path = os.path.join(project_path, 'Data')
code_path = os.path.join(project_path, 'Code', 'PyPerceive', 'code')

project_path, data_path, code_path
# change directory to code path???
os.chdir(code_path)
os.getcwd()

'c:\\Users\\jebe12\\Research\\PyPerceive_Project\\Code\\PyPerceive\\code'

## 1. Load Data using MainClass2

In [5]:
import PerceiveImport.classes.MainClass2 as mainclass
importlib.reload(mainclass)

<module 'PerceiveImport.classes.MainClass2' from 'c:\\Users\\jebe12\\Research\\PyPerceive_Project\\Code\\PyPerceive\\code\\PerceiveImport\\classes\\MainClass2.py'>

In [6]:
# define an example instance and fill in the values of the dataclass PerceiveData 
# choose the values you are interested in analyzing further

sub017 = mainclass.PerceiveData(
    sub = "017", 
    incl_modalities=["Survey"],
    incl_session = ["PostOp"],
    incl_condition =["M0S0"],
    #incl_task = ["RestBSSuRingR", "RestBSSuRingL"]
    )

# Funktion:
# Eingabe nur einzelner Strings, nicht mehrfacher Str in Liste
# bei keiner Eingabe der letzten attributes, wird default aus allen Möglichkeiten herausgegeben
# funktioniert nur bei vollständiger metadata Tabelle und nur, wenn filenames in perceiveFilenames == filenames in directory (nicht immer der Fall, vor allem der Anfang sub-017. sub017)
# d.h. metadata vorher überprüfen: neue xlsx mit matfiles in directory + paths (paths enthalten Info zB 3MFU) -> händisch copy paste in metadata Tabelle und vervollständigen


# Pitfalls:
# bei mehreren Eingaben, funktioniert immer nur der 1.Input -> zB incl_session = ["FU3M", "PostOp", "FU12M"] -> nur FU3M funktioniert, Rest ist leer
# wenn nichts eingegeben wurde, funktioniert immer nur der 1.Input der Default Liste! -> zB default Liste ["PostOp", "FU3M", ] -> nur PostOp funktioniert, Rest ist leer
# missing function: if one attribute is left out, the connection to the next Class is missing ...  
# missing function: can´t select multiple inputs simultaneously, optimal would be: input List -> outcome: concatenated dataframes of multiple DF

# neue Ziele: 

MetadataClass has been loaded


c:\Users\jebe12\anaconda3\envs\pyPerceive_dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [9]:
sub017.Survey.PostOp.M0S0.metadata_selection

,report,perceiveFilename,task,session,condition,stimState,run,badChannels,maxAmp,stimPeriodRestZeroPre,...,stimPeriodRestTwo,stimPeriodRestThree,stimPeriodRestMax,stimPeriodRestZeroPost,stimPeriodMovZero,stimPeriodMovOne,stimPeriodMovTwo,stimPeriodMovThree,stimPeriodMovMax,path_to_perceive


In [8]:
# Beispiel: ich möchte von sub 017, alle Survey files für jeden Timepoint (session) 
# erstelle je 1 DF für jede session

sub017_PostOp = mainclass.PerceiveData(
    sub = "017", 
    incl_modalities=["Survey"],
    incl_session = ["PostOp"],
    #incl_condition =["M0S0"],
    #incl_task = ["RestBSSuRingR", "RestBSSuRingL"]
    )

sub017_FU3M = mainclass.PerceiveData(
    sub = "017", 
    incl_modalities=["Survey"],
    incl_session = ["FU3M"],
    #incl_condition =["M0S0"],
    #incl_task = ["RestBSSuRingR", "RestBSSuRingL"]
    )

sub017_FU12M = mainclass.PerceiveData(
    sub = "017", 
    incl_modalities=["Survey"],
    incl_session = ["FU12M"],
    #incl_condition =["M0S0"],
    #incl_task = ["RestBSSuRingR", "RestBSSuRingL"]
    )    

sub017_dict = {
    "017PostOp_DF": sub017_PostOp.Survey.PostOp.metadata_selection, 
    "017PostOp_paths":sub017_PostOp.Survey.PostOp.matpath_list,
    "017FU3M_DF": sub017_FU3M.Survey.FU3M.metadata_selection, 
    "017FU3M_paths": sub017_FU3M.Survey.FU3M.matpath_list,
    "017FU12M_DF": sub017_FU12M.Survey.FU12M.metadata_selection, 
    "017FU12M_paths": sub017_FU12M.Survey.FU12M.matpath_list
}

#sub017_dict["017PostOp_DF"]#, sub017_dict["017FU3M_DF"], sub017_dict["017FU12M_DF"]
sub017_dict["017PostOp_DF"]

c:\Users\jebe12\anaconda3\envs\pyPerceive_dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jebe12\anaconda3\envs\pyPerceive_dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jebe12\anaconda3\envs\pyPerceive_dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


MetadataClass has been loaded
MetadataClass has been loaded
MetadataClass has been loaded


,report,perceiveFilename,task,session,condition,stimState,run,badChannels,maxAmp,stimPeriodRestZeroPre,...,stimPeriodRestTwo,stimPeriodRestThree,stimPeriodRestMax,stimPeriodRestZeroPost,stimPeriodMovZero,stimPeriodMovOne,stimPeriodMovTwo,stimPeriodMovThree,stimPeriodMovMax,path_to_perceive
0,sub-017_Report_Json_Session_Report_20210414T13...,sub-017_ses-2021041410364999_run-LMTD202104141...,RestBSSuRingL,PostOp,M1S0,Off,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
1,sub-017_Report_Json_Session_Report_20210414T13...,sub-017_ses-2021041410364999_run-LMTD202104141...,RestBSSuSegmIntraL,PostOp,M1S0,Off,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
2,sub-017_Report_Json_Session_Report_20210414T13...,sub-017_ses-2021041410364999_run-LMTD202104141...,RestBSSuSegmInterL,PostOp,M1S0,Off,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
3,sub-017_Report_Json_Session_Report_20210414T13...,sub-017_ses-2021041410364999_run-LMTD202104141...,RestBSSuRingR,PostOp,M1S0,Off,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
4,sub-017_Report_Json_Session_Report_20210414T13...,sub-017_ses-2021041410364999_run-LMTD202104141...,RestBSSuSegmIntraR,PostOp,M1S0,Off,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
5,sub-017_Report_Json_Session_Report_20210414T13...,sub-017_ses-2021041410364999_run-LMTD202104141...,RestBSSuSegmInterR,PostOp,M1S0,Off,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
